# Практическое задание 7

Домашняя работа начинается примерно посередине ноутбука

In [1]:
import numpy as np
import pandas as pd

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

from sklearn.model_selection import train_test_split

In [2]:
# Загрузим набор данных

df = pd.read_csv('./freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [3]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())

Unique values of ClaimInd: [0]


In [4]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [5]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [6]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [7]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


In [8]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


In [9]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [10]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [11]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

In [12]:
df = df.select_dtypes(exclude=['object'])

In [13]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)

In [14]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [15]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [16]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "12.0.2" 2019-07-16; Java(TM) SE Runtime Environment (build 12.0.2+10); Java HotSpot(TM) 64-Bit Server VM (build 12.0.2+10, mixed mode, sharing)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/5f/qdv8zptn4zd992t8d9d38wvw0000gn/T/tmpah2nl33n
  JVM stdout: /var/folders/5f/qdv8zptn4zd992t8d9d38wvw0000gn/T/tmpah2nl33n/h2o_fdima_started_from_python.out
  JVM stderr: /var/folders/5f/qdv8zptn4zd992t8d9d38wvw0000gn/T/tmpah2nl33n/h2o_fdima_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_fdima_xrrds0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


# Домашняя работа

Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных. 

Подобрать необходимое распределение и тип связи, при необходимости ознакомиться с документацией H20. 

Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков или функции от них и т.д.). 

Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке), проанализировать вероятные проблемы. 

Предложить способы их решения и/или попробовать их решить, улучшив результат.

In [17]:
# Разбиение датасета на train/val/test

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_test_ind, y_test_ind, test_size=0.5, random_state=1)

In [18]:
# Преобразование в H2O-Frame

h2o_train = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [19]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

h2o_train['ClaimInd'] = h2o_train['ClaimInd'].asfactor()
h2o_valid['ClaimInd'] = h2o_valid['ClaimInd'].asfactor()
h2o_test['ClaimInd'] = h2o_test['ClaimInd'].asfactor()

In [20]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_bin = H2OGeneralizedLinearEstimator(family='binomial', nfolds=5)

glm_bin.train(y='ClaimInd', 
              x=h2o_train.names[1:-1], 
              training_frame=h2o_train, 
              validation_frame=h2o_valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [21]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_bin.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 2.368E-5 )",20,20,3,py_1_sid_a3ae


In [22]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_bin.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.54542863,0.07844003,0.5572271,0.44817927,0.51773185,0.53936714,0.6646379
1,auc,0.56356305,0.009662209,0.5729706,0.5691177,0.5476904,0.56309086,0.56494564
2,aucpr,0.114547424,0.0052823583,0.113373324,0.1175144,0.10579905,0.11862023,0.11743012
3,err,0.45457137,0.07844003,0.44277292,0.55182076,0.48226818,0.46063286,0.33536208
4,err_count,7327.2,1253.8638,7211.0,8865.0,7765.0,7395.0,5400.0
5,f0point5,0.1349569,0.0050801374,0.13483508,0.13226433,0.12803598,0.1392417,0.14040738
6,f1,0.1876993,0.004885338,0.18859008,0.18944865,0.1803019,0.19365391,0.18650196
7,f2,0.30951205,0.020689385,0.31362274,0.3337414,0.30467355,0.31786942,0.2776532
8,lift_top_group,1.4256305,0.41451067,1.4742525,1.9590826,0.9190518,1.1224873,1.6532778
9,logloss,0.31125897,0.0068854704,0.30352572,0.31158763,0.31100103,0.32217205,0.3080085


In [23]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_bin._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.534238,-2.279643
1,LicAge,-0.000312,-0.049906
2,Gender,0.010311,0.004998
3,MariStat,-0.066143,-0.023827
4,DrivAge,-0.001085,-0.016265
5,HasKmLimit,-0.366059,-0.114316
6,BonusMalus,0.006341,0.097335
7,OutUseNb,0.061289,0.042639
8,RiskArea,0.009015,0.019975
9,VehUsage_Private,-0.146273,-0.069170


In [24]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_bin.coef_norm()

for x in range(len(glm_bin.cross_validation_models())):
    pmodels[x] = glm_bin.cross_validation_models()[x].coef_norm()
    
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-2.27964,-2.26899,-2.27985,-2.28115,-2.29418,-2.27567
LicAge,-0.04991,-0.01580,-0.07439,-0.02828,-0.06090,-0.06795
Gender,0.00500,0.00347,0.00000,0.01646,0.00355,0.00239
MariStat,-0.02383,-0.02751,-0.02208,-0.02304,-0.02672,-0.02260
DrivAge,-0.01626,-0.04504,0.01444,-0.03348,-0.07319,-0.01971
HasKmLimit,-0.11432,-0.11230,-0.11196,-0.12817,-0.10831,-0.11224
BonusMalus,0.09734,0.09601,0.10202,0.10777,0.09798,0.08006
OutUseNb,0.04264,0.05014,0.03803,0.04862,0.03830,0.03815
RiskArea,0.01997,0.02052,0.01607,0.02713,0.01843,0.01860
VehUsage_Private,-0.06917,-0.04925,-0.07030,-0.05050,-0.06242,-0.08111


In [25]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

train_pred = glm_bin.predict(h2o_train).as_data_frame()
valid_pred = glm_bin.predict(h2o_valid).as_data_frame()
test_pred = glm_bin.predict(h2o_test).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [26]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [27]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

def show_scores(y_true, y_pred):
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}\n')
    print(f'F1-score: {accuracy_score(y_true, y_pred)}\n')
    print(f'Confusion matrix:\n{confusion_matrix(y_true, y_pred)}')

In [28]:
show_scores(y_train_ind, train_pred['predict'])

Accuracy: 0.5789623858674077

F1-score: 0.5789623858674077

Confusion matrix:
[[42807 30166]
 [ 3773  3862]]


In [29]:
show_scores(y_valid_ind, valid_pred['predict'])

Accuracy: 0.5829329010594569

F1-score: 0.5829329010594569

Confusion matrix:
[[9243 6421]
 [ 783  826]]


In [30]:
show_scores(y_test_ind, test_pred['predict'])

Accuracy: 0.5762996410790784

F1-score: 0.5762996410790784

Confusion matrix:
[[9126 6523]
 [ 796  829]]


Какие проблемы вы здесь видите? Как можно улучшить данный результат?

Можно попробовать избавиться от дизбаланса классов. Возможно, лучше почистить данные. Также можно попробовать другие модели.